# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [1]:
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
os.getcwd()

'C:\\Users\\mci\\Desktop\\Guided_Capstone1\\Step5'

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [2]:
path='C:\\Users\\mci\\Desktop\\Guided_Capstone1\\Step 2\\data'
os.chdir(path)
os.getcwd()

'C:\\Users\\mci\\Desktop\\Guided_Capstone1\\Step 2\\data'

## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [3]:
bmd = pd.read_csv('step3_output.csv')
bmd = bmd.drop('Unnamed: 0', axis=1)
bmd.head()

,Name,state,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Alyeska Resort,Alaska,3939,2500,250,1,0.0,0,2,2,...,1610.0,113.0,150.0,60.0,669.0,65.0,85.0,150.0,550.0,0
1,Eaglecrest Ski Area,Alaska,2600,1540,1200,0,0.0,0,0,0,...,640.0,60.0,45.0,44.0,350.0,47.0,53.0,90.0,0.0,0
2,Hilltop Ski Area,Alaska,2090,294,1796,0,0.0,0,0,0,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,0
3,Arizona Snowbowl,Arizona,11500,2300,9200,0,0.0,1,0,2,...,777.0,104.0,122.0,81.0,260.0,89.0,89.0,122.0,0.0,1
4,Sunrise Park Resort,Arizona,11100,1800,9200,0,0.0,0,1,2,...,800.0,80.0,115.0,49.0,250.0,74.0,78.0,104.0,80.0,1


In [4]:
dummy = pd.get_dummies(bmd['state'])
df = pd.concat([bmd,dummy], axis=1) 
del df['state']
df.head()

,Name,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,triple,...,Rhode Island,South Dakota,Tennessee,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,Alyeska Resort,3939,2500,250,1,0.0,0,2,2,0,...,0,0,0,0,0,0,0,0,0,0
1,Eaglecrest Ski Area,2600,1540,1200,0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hilltop Ski Area,2090,294,1796,0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Arizona Snowbowl,11500,2300,9200,0,0.0,1,0,2,2,...,0,0,0,0,0,0,0,0,0,0
4,Sunrise Park Resort,11100,1800,9200,0,0.0,0,1,2,3,...,0,0,0,0,0,0,0,0,0,0


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [5]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
x = df.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.AdultWeekend 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(x)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(x) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [6]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [7]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [8]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred=model.predict(X_test)
y_pred

array([ 58.62516431,  90.35953931,  79.28532056,  76.75602369,
        69.85075025,  76.84879713,  44.66032056,  46.92448072,
        93.96598463,  83.92203931,  50.73893385,  79.23160963,
        51.2975276 ,  71.02555494,  62.65592603,  92.6202815 ,
        37.456219  ,  72.81950025,  62.87418775,  41.43375806,
       113.41276197,  36.78239088,  53.29508619,  49.32731275,
        48.27750806,  60.67301588,  76.48844556,  70.52946119,
        63.88883619,  66.56461744,  35.36930494,  60.61539869,
        80.40055494,  56.94742994,  75.37711744,  49.65055494,
        22.91227369,  28.85660963,  61.36149244,  36.94352369,
        47.62321119,  61.63688306,  86.26578931,  72.35563306,
        72.7584651 ,  59.643719  ,  62.00651197, 166.11930494,
        38.21891431,  61.49430494,  68.36539869,  58.28336744,
        53.37907056,  45.72086744,  59.15543775,  42.61930494,
        86.09586744,  38.8234065 , 119.66422681, 119.40836744,
        55.81852369,  54.44010572,  41.88492994,  80.16

## Review Model Outcomes — Iterate over additional models as needed

In [9]:
# You might want to use the explained_variance_score() and explained_variance_score() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model

In [10]:
from sklearn.metrics import explained_variance_score,mean_absolute_error
evs=explained_variance_score(y_test, y_pred)
mean=explained_variance_score(y_test, y_pred)
evs, mean

(0.9354241293809651, 0.9354241293809651)

**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [11]:
print(model.intercept_)

64.11149243901306


**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [12]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
# pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])
coef = pd.DataFrame(abs(lm.coef_), x.columns, columns=['Coefficient'])
coef.sort_values('Coefficient', ascending=False).head(10)

,Coefficient
total_chairs,2.554157e+13
fastQuads,9.682867e+12
surface,9.072121e+12
double,7.994689e+12
triple,7.131812e+12
quad,5.780069e+12
fastSixes,2.870491e+12
trams,2.466406e+12
New York,2.434230e+12
Michigan,2.297252e+12


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [13]:
bmd1 = bmd.drop('state', axis=1) 
X = bmd1.drop(['Name','AdultWeekend'], axis=1) 
y = bmd1.AdultWeekend 
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
y = y.ravel() 
X2_train, X2_test, y2_train, y2_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

In [14]:
model_2 = lm.fit(X2_train,y2_train)
y2_pred = model_2.predict(X2_test)
coef = pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])
coef.sort_values('Coefficient', ascending=False).head(10)

,Coefficient
AdultWeekday,19.954000
base_elev,15.135123
summit_elev,13.083599
clusters,6.144047
vertical_drop,5.289959
Runs,1.725803
surface,1.528398
triple,1.458477
averageSnowfall,1.277009
quad,1.188231


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [15]:
bmd2 = bmd.drop(['state','summit_elev','base_elev'], axis=1) 
X = bmd2.drop(['Name','AdultWeekend'], axis=1) 
y = bmd2.AdultWeekend 
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
y = y.ravel()
X3_train, X3_test, y3_train, y3_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

In [16]:
model_3 = lm.fit(X3_train,y3_train)
y3_pred = model_3.predict(X3_test)
coef = pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])
coef.sort_values('Coefficient', ascending=False).head(10)

,Coefficient
AdultWeekday,20.128806
clusters,2.647871
vertical_drop,2.064305
triple,1.450975
Runs,1.361873
surface,1.250484
quad,1.239127
daysOpenLastYear,1.201522
averageSnowfall,0.987277
fastQuads,0.767482


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0. |  |-|
| Model 2. | 0.| |'state'|
| Model 3. | 0. | |'state','summit_elev','base_elev'|

Model Selection:

In [17]:
evs_m1 = explained_variance_score(y_test, y_pred)
mean_m1 = mean_absolute_error(y_test, y_pred)


evs_m2 = explained_variance_score(y2_test, y2_pred)
mean_m2 = mean_absolute_error(y2_test, y2_pred)


evs_m3 = explained_variance_score(y3_test, y3_pred)
mean_m3 = mean_absolute_error(y3_test, y3_pred)


In [18]:
review = {'Model':['Model 1','Model 2', 'Model 3'],'Explained Variance':[evs_m1,evs_m2,evs_m3],'Mean Absolute Error':[mean_m1,mean_m2,mean_m3],'Features Dropped':['-','state','state, summit_elev, base_elev']}

models = pd.DataFrame(review).set_index('Model')
models

,Explained Variance,Mean Absolute Error,Features Dropped
Model,,,
Model 1,0.935424,5.006802,-
Model 2,0.929575,5.272954,state
Model 3,0.936530,5.094973,"state, summit_elev, base_elev"
